In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16

import matplotlib.pyplot as plt
import random
import zipfile
import pandas as pd
%matplotlib inline


f = zipfile.ZipFile("./drive/My Drive/AOI/train_images.zip")
f.extractall("./datasets")

df = pd.read_csv("./drive/My Drive/AOI/train.csv")

In [5]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.layers import BatchNormalization

cnn = VGG16(include_top=False, input_shape=(256, 256, 3))

for l in cnn.layers:
    l.trainable = False

x = BatchNormalization()(cnn.output)
x = Flatten()(x)
x = Dense(64, activation="relu")(x)
out = Dense(6, activation="softmax")(x)

model2 = Model(inputs=cnn.input, outputs=out)

In [ ]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy

model2.compile(loss=SparseCategoricalCrossentropy(),
               optimizer="adam",
               metrics=["accuracy"])

In [8]:
import numpy as np
from sklearn.model_selection import train_test_split

imagePaths = np.array("./datasets/train_images/" + df["ID"])
label = np.array(df["Label"])

# 圖片, 編號, 圖片, 編號
x_train, x_test, y_train, y_test = train_test_split(imagePaths,
                                                    label,
                                                    test_size=0.2)

[0 1 1 ... 3 0 5]
(2528,)

['./datasets/train_images/train_00000.png'
 './datasets/train_images/train_00001.png'
 './datasets/train_images/train_00002.png' ...
 './datasets/train_images/train_02525.png'
 './datasets/train_images/train_02526.png'
 './datasets/train_images/train_02527.png']


In [93]:
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing.image import load_img

def dataflow (x_data, y_data, batch=20):
    
    x_len = len(x_data)
    batch_index = np.random.choice(x_len, batch)
    
    x_batch, y_batch = [], []
    for x in batch_index:
        img = load_img(x_train[x], target_size=(256, 256))
        img = preprocess_input(np.array(img))
        x_batch.append(img)
        
        y_batch.append(y_data[x])
    
    return x_batch, y_batch

In [51]:
# 訓練個 10 次
# 這個可以一直按 = 10 + 10 + 10 + ... + n 次 訓練

for i in range(10):
    print("-" * 10, "這是第", i+1, "次訓練")
    x_trn_batch, y_trn_batch = dataflow(x_train, y_train)
    result1 = model2.train_on_batch(x_trn_batch, y_trn_batch)  ## train on batch

    # 驗證資料的 batch 要高點比較好
    x_tet_batch, y_tet_batch = dataflow(x_test, y_test, t='test')
    result2 = model2.test_on_batch(x_tet_batch, y_tet_batch)  ## test on batch

    print("[Train]:", result1)
    print("[valid]:", result2)

---------- 這是第 1 次訓練


NameError: name 'x_train' is not defined

In [52]:
# 拿 10% 的 200 張 算 evaluate = ['loss', 'acc']
tx, ty = dataflow(x_test, y_test, 200)
model2.evaluate(tx, ty)

# 拿 10% 的驗證資料看機率值
pred = model2.predict(x_prepro02)
# pred

NameError: name 'x_test' is not defined